In [ ]:
!pip install dm_control
# !pip install pink-noise-rl
!pip install wandb

In [ ]:
import torch as th
device = th.device("cuda" if th.cuda.is_available() else "cpu")
print(device)

In [ ]:
import gym
from gym import spaces

from dm_control import suite
from dm_env import specs


def convert_dm_control_to_gym_space(dm_control_space):
    r"""Convert dm_control space to gym space. """
    if isinstance(dm_control_space, specs.BoundedArray):
        space = spaces.Box(low=dm_control_space.minimum, 
                           high=dm_control_space.maximum, 
                           dtype=dm_control_space.dtype)
        assert space.shape == dm_control_space.shape
        return space
    elif isinstance(dm_control_space, specs.Array) and not isinstance(dm_control_space, specs.BoundedArray):
        space = spaces.Box(low=-float('inf'), 
                           high=float('inf'), 
                           shape=dm_control_space.shape, 
                           dtype=dm_control_space.dtype)
        return space
    elif isinstance(dm_control_space, dict):
        space = spaces.Dict({key: convert_dm_control_to_gym_space(value)
                             for key, value in dm_control_space.items()})
        return space


class DMSuiteEnv(gym.Env):
    def __init__(self, domain_name, task_name, task_kwargs=None, environment_kwargs=None, visualize_reward=False):
        self.env = suite.load(domain_name, 
                              task_name, 
                              task_kwargs=task_kwargs, 
                              environment_kwargs=environment_kwargs, 
                              visualize_reward=visualize_reward)
        self.metadata = {'render.modes': ['human', 'rgb_array'],
                         'video.frames_per_second': round(1.0/self.env.control_timestep())}
        print(self.env.observation_spec())
        self.observation_space = convert_dm_control_to_gym_space(self.env.observation_spec())
        print(self.observation_space)
        print("________________________")
        print(self.env.action_spec())
        self.action_space = convert_dm_control_to_gym_space(self.env.action_spec())
        print(self.action_space)
        self.viewer = None
    
    def seed(self, seed):
        return self.env.task.random.seed(seed)
    
    def step(self, action):
        timestep = self.env.step(action)
        observation = timestep.observation
        reward = timestep.reward
        done = timestep.last()
        info = {}
        truncated = False
        return observation, reward, done, info
    
    def reset(self):
        timestep = self.env.reset()
        return timestep.observation
    
    def render(self, mode='human', **kwargs):
        if 'camera_id' not in kwargs:
            kwargs['camera_id'] = 0  # Tracking camera
        use_opencv_renderer = kwargs.pop('use_opencv_renderer', False)
        
        img = self.env.physics.render(**kwargs)
        if mode == 'rgb_array':
            return img
        elif mode == 'human':
            if self.viewer is None:
                if not use_opencv_renderer:
                    from gym.envs.classic_control import rendering
                    self.viewer = rendering.SimpleImageViewer(maxwidth=1024)
                else:
                    from . import OpenCVImageViewer
                    self.viewer = OpenCVImageViewer()
            self.viewer.imshow(img)
            return self.viewer.isopen
        else:
            raise NotImplementedError

    def close(self):
        if self.viewer is not None:
            self.viewer.close()
            self.viewer = None
        return self.env.close()

In [ ]:
import numpy as np
from numpy.fft import irfft, rfftfreq


def powerlaw_psd_gaussian(exponent, size, fmin=0, rng=None):
    """Gaussian (1/f)**beta noise.

    Based on the algorithm in:
    Timmer, J. and Koenig, M.:
    On generating power law noise.
    Astron. Astrophys. 300, 707-710 (1995)

    Normalised to unit variance

    Parameters:
    -----------

    exponent : float
        The power-spectrum of the generated noise is proportional to

        S(f) = (1 / f)**beta
        flicker / pink noise:   exponent beta = 1
        brown noise:            exponent beta = 2

        Furthermore, the autocorrelation decays proportional to lag**-gamma
        with gamma = 1 - beta for 0 < beta < 1.
        There may be finite-size issues for beta close to one.

    shape : int or iterable
        The output has the given shape, and the desired power spectrum in
        the last coordinate. That is, the last dimension is taken as time,
        and all other components are independent.

    fmin : float, optional
        Low-frequency cutoff.
        Default: 0 corresponds to original paper.

        The power-spectrum below fmin is flat. fmin is defined relative
        to a unit sampling rate (see numpy's rfftfreq). For convenience,
        the passed value is mapped to max(fmin, 1/samples) internally
        since 1/samples is the lowest possible finite frequency in the
        sample. The largest possible value is fmin = 0.5, the Nyquist
        frequency. The output for this value is white noise.

    rng : np.random.Generator, optional
        Random number generator (for reproducibility). If not passed, a new
        random number generator is created by calling
        `np.random.default_rng()`.


    Returns
    -------
    out : array
        The samples.


    Examples:
    ---------

    >>> # generate 1/f noise == pink noise == flicker noise
    >>> import colorednoise as cn
    >>> y = cn.powerlaw_psd_gaussian(1, 5)
    """

    # Make sure size is a list so we can iterate it and assign to it.
    try:
        size = list(size)
    except TypeError:
        size = [size]

    # The number of samples in each time series
    samples = size[-1]

    # Calculate Frequencies (we asume a sample rate of one)
    # Use fft functions for real output (-> hermitian spectrum)
    f = rfftfreq(samples)

    # Validate / normalise fmin
    if 0 <= fmin <= 0.5:
        fmin = max(fmin, 1./samples)    # Low frequency cutoff
    else:
        raise ValueError("fmin must be chosen between 0 and 0.5.")

    # Build scaling factors for all frequencies
    s_scale = f
    ix = np.sum(s_scale < fmin)   # Index of the cutoff
    if ix and ix < len(s_scale):
        s_scale[:ix] = s_scale[ix]
    s_scale = s_scale**(-exponent/2.)

    # Calculate theoretical output standard deviation from scaling
    w = s_scale[1:].copy()
    w[-1] *= (1 + (samples % 2)) / 2.    # correct f = +-0.5
    sigma = 2 * np.sqrt(np.sum(w**2)) / samples

    # Adjust size to generate one Fourier component per frequency
    size[-1] = len(f)

    # Add empty dimension(s) to broadcast s_scale along last
    # dimension of generated random power + phase (below)
    dims_to_add = len(size) - 1
    s_scale = s_scale[(None,) * dims_to_add + (Ellipsis,)]

    # Generate scaled random power + phase
    if rng is None:
        rng = np.random.default_rng()
    sr = rng.normal(scale=s_scale, size=size)
    si = rng.normal(scale=s_scale, size=size)

    # If the signal length is even, frequencies +/- 0.5 are equal
    # so the coefficient must be real.
    if not (samples % 2):
        si[..., -1] = 0
        sr[..., -1] *= np.sqrt(2)    # Fix magnitude

    # Regardless of signal length, the DC component must be real
    si[..., 0] = 0
    sr[..., 0] *= np.sqrt(2)    # Fix magnitude

    # Combine power + corrected phase to Fourier components
    s = sr + 1J * si

    # Transform to real time series & scale to unit variance
    y = irfft(s, n=samples, axis=-1) / sigma

    return y

In [ ]:
class ColoredNoiseProcess():
    """Colored noise implemented as a process that allows subsequent samples.
    Implemented as a buffer; every "chunksize[-1]" items, a cut to a new time series starts.
    """

    def __init__(self, beta=1, scale=1, chunksize=32768, largest_wavelength=256, rng=None):
        self.beta = beta
        if largest_wavelength is None:
            self.minimum_frequency = 0
        else:
            self.minimum_frequency = 1 / largest_wavelength
        self.scale = scale
        self.rng = rng

        # The last component of chunksize is the time index
        try:
            self.chunksize = list(chunksize)
        except TypeError:
            self.chunksize = [chunksize]
        self.time_steps = self.chunksize[-1]

        # Set first time-step such that buffer will be initialized
        self.idx = self.time_steps

    def sample(self):
        self.idx += 1    # Next time step

        # Refill buffer if depleted
        if self.idx >= self.time_steps:
            self.buffer = powerlaw_psd_gaussian(
                exponent=self.beta, size=self.chunksize, fmin=self.minimum_frequency, rng=self.rng)
            self.idx = 0

        return self.scale * self.buffer[..., self.idx]

In [ ]:
from stable_baselines3.common.distributions import SquashedDiagGaussianDistribution
from stable_baselines3.common.noise import ActionNoise

class SquashedDiagCNDistribution(SquashedDiagGaussianDistribution):
    """
    Colored Noise distribution with diagonal covariance matrix, followed by a squashing function (tanh) to ensure
    bounds. Used for Soft Actor-Critic with colored noise exploration in lieu of SquashedDiagGaussianDistribution.

    :param action_dim: Dimension of the action space.
    :param epsilon: small value to avoid NaN due to numerical imprecision.
    """

    def __init__(self, action_dim: int, beta: np.ndarray, seq_len, epsilon: float = 1e-6, rng=None):
        super().__init__(action_dim, epsilon)
        self.cn_processes = [ColoredNoiseProcess(beta=b, chunksize=seq_len, largest_wavelength=None, rng=rng)
                             for b in beta]

    def sample(self) -> th.Tensor:
        cn_sample = th.tensor([cnp.sample() for cnp in self.cn_processes]).float()
        cn_sample = cn_sample.to(device)
        self.gaussian_actions = self.distribution.mean + self.distribution.stddev*cn_sample
        return th.tanh(self.gaussian_actions)
    
    
class ColoredActionNoise(ActionNoise):
    """Action noise using colored noise processes (independent for each action dimension)."""
    def __init__(self, beta, sigma, seq_len, action_dim=None, rng=None):
        super().__init__()
        assert (action_dim is not None) == np.isscalar(beta), \
            "`action_dim` has to be specified if and only if `beta` is a scalar."

        self.sigma = np.full(action_dim or len(beta), sigma) if np.isscalar(sigma) else np.asarray(sigma)

        if np.isscalar(beta):
            self.beta = beta
            self.gen = ColoredNoiseProcess(beta=self.beta, scale=self.sigma, size=(action_dim, seq_len), rng=rng)
        else:
            self.beta = np.asarray(beta)
            self.gen = [ColoredNoiseProcess(beta=b, scale=s, size=seq_len, rng=rng)
                        for b, s in zip(self.beta, self.sigma)]

    def __call__(self) -> np.ndarray:
        return self.gen.sample() if np.isscalar(self.beta) else np.asarray([g.sample() for g in self.gen])

    def __repr__(self) -> str:
        return f"ColoredActionNoise(beta={self.beta}, sigma={self.sigma})"

In [ ]:
class GenEnv():
    """Generate gym environment from string with new seed each time"""
    def __init__(self, seed, sparse_reward=False, tonic=False):
        self.env = gym.make("MountainCarContinuous-v0")
        self.seed = seed
        self.sparse_reward = sparse_reward
        self.tonic = tonic

    def __call__(self, get_policy=False):
        policy = "MlpPolicy"
        env = gym.make("MountainCarContinuous-v0")

        if self.sparse_reward:
            env.step_ = env.step

            def step(env, a):
                obs, _, done, info = env.step_(a)
                return obs, int(info['goal_achieved']), done, info

            env.step = MethodType(step, env)

        self.seed += 1    # Change seed such that it is different each time the environment is created

        if get_policy:
            return env, policy
        return env

In [ ]:
from stable_baselines3.common.callbacks import BaseCallback

class MonitorCallback(BaseCallback):
    """Simple SB3 callback for monitoring episode returns and lengths. Also shows a progress bar."""
    def __init__(self, gen_env, save=False, eval_every=10_000, n_eval=5):
        super().__init__()
        self.save = save
        self.env = gen_env()
        self.eval_every = eval_every    # Every eval_every interactions, commence evaluation step
        self.n_eval = n_eval            # Number of evaluation episodes per evaluation step
        self.episode_returns = []
        self.episode_lengths = []
        self.evaluation_returns = []
        self.monitor = {'episode_returns': self.episode_returns, 'episode_lengths': self.episode_lengths,
                        'evaluation_returns': self.evaluation_returns}

    def _on_training_start(self):
        self.n_actions = self.training_env.action_space.shape[-1]
        self.episode_returns.append(0)
        self.episode_lengths.append(0)

        # Progress bar

    def _on_training_end(self):
        # Save data and close progress bar
        pass

    def _on_step(self):

        # Store reward
        self.episode_returns[-1] += self.locals['rewards'].item()
        self.episode_lengths[-1] += 1
        if self.locals['dones'].item():
            self.episode_returns.append(0)
            self.episode_lengths.append(0)

        # Evaluation Rollouts
        


class ScheduledColoredNoise(MonitorCallback):
    """Linear scheduling from colors[0] to colors[1]"""
    def __init__(self, gen_env, save=False, colors=(2, 0), method='linear', noise_scale=0.3, len_rollout=10_000,
                 rng=None, **monitor_kwargs):
        super().__init__(gen_env, save, **monitor_kwargs)
        self.colors = colors
        self.method = method              # 'linear' or 'atanh' scheduling
        self.noise_scale = noise_scale    # Colored noise process std
        self.len_rollout = len_rollout    # Number of interactions per rollout
        self.i = 0                        # Iteration counter
        self.rng = rng                    # Random number generator

        # Monitoring
        self.chosen_colors = []
        self.monitor['colors'] = self.chosen_colors

    def _on_training_start(self):
        super()._on_training_start()

        # Initialize agent noise
        self.beta = self.update_beta()

    def set_beta(self, beta, seq_len):
        if isinstance(self.model, SAC):
            # State-dependent action noise is a bit more difficult
            self.model.actor.action_dist = SquashedDiagCNDistribution(
                self.n_actions, beta, seq_len, rng=self.rng)
        else:
            # For TD3 or other non-state dependent action noise agents
            self.model.action_noise = ColoredActionNoise(
                beta, self.noise_scale * np.ones(self.n_actions), seq_len, rng=self.rng)

    def update_beta(self):
        """Select new beta parameter (randomly)"""
        # Calculate and apply beta
        x = self.num_timesteps / self.locals['total_timesteps']
        if self.method == 'linear':
            beta = (1 - x)*self.colors[0] + x*self.colors[1]
        elif self.method == 'atanh':
            beta = np.clip(np.arctanh(min(1 - x, 1 - 1e-6)), 0, 4)
        elif self.method == 'cosine':
            beta = self.colors[1] + 0.5 * (self.colors[0]-self.colors[1]) * (1+math.cos(math.pi*x))
        beta *= np.ones(self.n_actions)
        self.set_beta(beta, self.len_rollout)

        # Monitoring
        self.chosen_colors.append(beta)

        return beta

    def _on_step(self):
        super()._on_step()

        # Check if rollout is finished
        i = self.num_timesteps // self.len_rollout
        if i == self.i:
            return
        self.i = i

        # Select new beta and change noise process
        self.beta = self.update_beta()

In [ ]:
def linear(x,start,stop):
    return (1-x)*start + x*stop

In [ ]:
import math
def cosine(x,start,stop):
    return stop + 0.5 * (start-stop) * (1+math.cos(math.pi*x))

In [ ]:
def atanh(x,start,stop):
    return np.clip(np.arctanh(min(1 - x, 1 - 1e-6)), 0, 4)

In [ ]:
import gymnasium as gym
import numpy as np
from stable_baselines3 import SAC
import time
from tqdm import tqdm

# Define a function to evaluate an episode
def evaluate_episode(model, env):
    obs,_ = env.reset()
    done = False
    total_reward = 0.0
    steps=0
    while steps<1000 and not done:
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, _ ,_= env.step(action)
        total_reward += reward
        steps+=1
    return total_reward

# Reproducibility
seed = 69
np.random.seed(seed)
th.manual_seed(seed)
rng = np.random.default_rng(seed)

# Initialize environment

gen_env = GenEnv(seed=420)
dir_ = '.'
noise_scale = 0.3
env = gen_env(get_policy=False)
ep = 100
rng = np.random.default_rng(seed = 3939)
action_dim = env.action_space.shape[-1]
seq_len = 1000

start_linear = 2
stop_linear = 2
start_atanh = 2
stop_atanh = 2
start_cosine = 2
stop_cosine = 2

# Initialize agents
model_linear = SAC("MlpPolicy", env, seed = seed)
model_atanh = SAC("MlpPolicy", env, seed = seed)
model_cosine = SAC("MlpPolicy", env, seed=seed)


# Training parameters
total_timesteps = 1000000
eval_frequency = 10000 # Evaluate every 104 interactions
eval_rollouts = 5

wandb.init(
    project="Pinkie",
    config = {
    "Total_timesteps": total_timesteps,
    "Eval_frequency": eval_frequency,
    "Eval_rollouts": eval_rollouts
    }
)

#Final average performances
avg_linear=0.0
avg_cosine=0.0
avg_atanh=0.0
final_linear=0.0
final_cosine=0.0
final_atanh=0.0

# Train agents with evaluation
timesteps_so_far = 0
# for timesteps_so_far in tqdm(range(0,total_timesteps,eval_frequency)):
while timesteps_so_far < total_timesteps:
    
    timesteps_so_far += eval_frequency
    x = timesteps_so_far/total_timesteps
    start_linear = stop_linear
    stop_linear = linear(x,2,0)
    callback_linear = ScheduledColoredNoise(
                gen_env, dir_, (start_linear, stop_linear), 'linear', noise_scale,
                ep, rng)
    
    t1 = time.time()
    # Train the default noise model
#     print(model_linear.actor.action_dist.gen.beta)
    model_linear.learn(total_timesteps=eval_frequency, log_interval = 1,callback = callback_linear )
#     print(model_linear.actor.action_dist.gen.beta)
    t2 = time.time()

    # Evaluate the default noise model
    mean_return_linear = 0.0
    for _ in range(eval_rollouts):
        mean_return_linear += evaluate_episode(model_linear, env)
    mean_return_linear /= eval_rollouts
    avg_linear+=mean_return_linear
    if(timesteps_so_far>=0.95*total_timesteps):
        final_linear+=mean_return_linear

    print(f"Return (Linear): {mean_return_linear}")
    print(f"Time taken (Linear Model): {t2 - t1:.2f} seconds")
    print(f"Timesteps: {timesteps_so_far}, Mean Return: {mean_return_linear}")
    
    
    start_cosine = stop_cosine
    stop_cosine = cosine(x,2,0)
    callback_cosine = ScheduledColoredNoise(
                gen_env, dir_, (start_cosine, stop_cosine), 'linear', noise_scale,
                ep, rng)
    
    t1 = time.time()
    # Train the default noise model
#     print(model_cosine.actor.action_dist.gen.beta)
    model_cosine.learn(total_timesteps=eval_frequency,log_interval = 1, callback = callback_cosine)
#     print(model_cosine.actor.action_dist.gen.beta)
    t2 = time.time()

    # Evaluate the default noise model
    mean_return_cosine = 0.0
    for _ in range(eval_rollouts):
        mean_return_cosine += evaluate_episode(model_cosine, env)
    mean_return_cosine /= eval_rollouts
    avg_cosine+=mean_return_cosine
    if(timesteps_so_far>=0.95*total_timesteps):
        final_cosine+=mean_return_cosine

    print(f"Return (Cosine): {mean_return_cosine}")
    print(f"Time taken (Cosine Model): {t2 - t1:.2f} seconds")
    print(f"Timesteps: {timesteps_so_far}, Mean Return: {mean_return_cosine}")
    
    start_atanh = stop_atanh
    stop_atanh = atanh(x,2,0)
    callback_atanh = ScheduledColoredNoise(
                gen_env, dir_, (start_atanh, stop_atanh), 'linear', noise_scale,
                ep, rng)
    
    t1 = time.time()
    # Train the default noise model
#     print(model_atanh.actor.action_dist.gen.beta)
    model_atanh.learn(total_timesteps=eval_frequency,log_interval = 1,callback = callback_atanh)
#     print(model_atanh.actor.action_dist.gen.beta)
    t2 = time.time()

    # Evaluate the default noise model
    mean_return_atanh = 0.0
    for _ in range(eval_rollouts):
        mean_return_atanh += evaluate_episode(model_atanh, env)
    mean_return_atanh /= eval_rollouts
    avg_atanh+=mean_return_atanh
    if(timesteps_so_far>=0.95*total_timesteps):
        final_atanh+=mean_return_atanh

    print(f"Return (Atanh): {mean_return_atanh}")
    print(f"Time taken (Atanh Model): {t2 - t1:.2f} seconds")
    print(f"Timesteps: {timesteps_so_far}, Mean Return: {mean_return_atanh}")
    
    wandb.log({
        "mean_return_linear": mean_return_linear,
        "timesteps_so_far": timesteps_so_far,
        "mean_return_cosine": mean_return_cosine,
        "mean_return_atanh": mean_return_atanh,
        "linear_beta": stop_linear,
        "cosine_beta": stop_cosine,
        "atanh_beta": stop_atanh
    })

avg_linear/=(total_timesteps/eval_frequency)
avg_cosine/=(total_timesteps/eval_frequency)
avg_atanh/=(total_timesteps/eval_frequency)

final_linear/=(0.05*total_timesteps/eval_frequency)
final_cosine/=(0.05*total_timesteps/eval_frequency)
final_atanh/=(0.05*total_timesteps/eval_frequency)

wandb.log({
    "final_linear": final_linear,
    "final_cosine": final_cosine,
    "final_atanh": final_atanh,
    "avg_linear": avg_linear,
    "avg_cosine": avg_cosine,
    "avg_atanh": avg_atanh
})

print("Mean:")
print(f"Linear:{avg_linear}           Cosine:{avg_cosine}             atanh:{avg_atanh}")
print("Final:")
print(f"Linear:{final_linear}           Cosine:{final_cosine}             atanh:{final_atanh}")